#### 结合归一化和正则化来优化网络模型结构，观察对比loss结果。


In [10]:
import torch
import torch.nn as nn
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split



# 定义超参数
LR = 1e-3
epochs = 1000 #训练轮次
BATCH_SIZE = 10 


# 1.生成随机样本
X, y = fetch_olivetti_faces(data_home='./face_data',return_X_y=True,shuffle=True)

# #拆分数据
# #局部样本训练模型（过拟合模型）
# #新样板数据模型表现不好（泛化能力差）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)


train_data_set = [(img,lbl) for img,lbl in zip(X_train, y_train)]
test_data_set = [(img,lbl) for img,lbl in zip(X_test, y_test)]


train_dl = torch.utils.data.DataLoader(train_data_set, batch_size=BATCH_SIZE, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_data_set, batch_size=BATCH_SIZE, shuffle=True)


In [ ]:
# 归一化&正则化模型
model = nn.Sequential(
    nn.Linear(4096, 256),  #输入层
    nn.BatchNorm1d(256),       #归一化
    nn.Dropout(p=0.2),      #正则化
    nn.ReLU(),     #隐藏层
    nn.Linear(256, 40)  #输出层
)

In [13]:


#损失函数和优化器
loss_fn = nn.CrossEntropyLoss() #交叉熵损失函数

#优化器（模型参数更新）
optimizer = torch.optim.SGD(model.parameters(), lr=LR) #SGD优化器


for epoch in range(epochs):
    for data, target in train_dl:
        #前向运算
        output = model(data.reshape(-1, 4096))
        #计算损失
        loss = loss_fn(output, target) #计算梯度
        #反向传播
        optimizer.zero_grad() #所有参数梯度清零
        loss.backward()  #计算梯度（参数.grad）
        optimizer.step() #更新参数
    print(f'Epoch:{epoch}, Loss:{loss.item()}')

Epoch:0, Loss:3.3886501789093018
Epoch:1, Loss:3.0822370052337646
Epoch:2, Loss:2.912524461746216
Epoch:3, Loss:2.6705965995788574
Epoch:4, Loss:2.558967113494873
Epoch:5, Loss:2.2611029148101807
Epoch:6, Loss:2.162395477294922
Epoch:7, Loss:2.172053813934326
Epoch:8, Loss:1.887542724609375
Epoch:9, Loss:2.067408800125122
Epoch:10, Loss:1.7134767770767212
Epoch:11, Loss:1.4878449440002441
Epoch:12, Loss:1.67191481590271
Epoch:13, Loss:1.847339391708374
Epoch:14, Loss:1.5335842370986938
Epoch:15, Loss:1.316829800605774
Epoch:16, Loss:1.4632930755615234
Epoch:17, Loss:1.1311115026474
Epoch:18, Loss:1.1728293895721436
Epoch:19, Loss:1.1863683462142944
Epoch:20, Loss:1.2721021175384521
Epoch:21, Loss:1.3038678169250488
Epoch:22, Loss:1.3553307056427002
Epoch:23, Loss:1.2136659622192383
Epoch:24, Loss:1.0665656328201294
Epoch:25, Loss:0.9330986142158508
Epoch:26, Loss:0.9201140403747559
Epoch:27, Loss:0.7724847197532654
Epoch:28, Loss:0.8778423070907593
Epoch:29, Loss:0.8754262924194336
Epo

In [17]:
# 测试
correct = 0
total = 0

model.eval()  #train(false)
with torch.no_grad(): #不计算梯度
    for data, target in test_dl:
        output = model(data.reshape(-1, 4096))
        _, predicted = torch.max(output, 1) #返回第一个维度的最大值张量
        total += target.size(0) #返回张量的样本量
        correct += (predicted == target).sum().item()
print(f'Accuracy: {correct/total*100}%')


Accuracy: 97.5%
